In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data/eda_dataset_imputed.csv')

app_train, test_1 = train_test_split(data, test_size=0.30, random_state=64)
app_test, app_validation = train_test_split(test_1, test_size=0.5, random_state=56)

In [3]:
train_labels = app_train['TARGET']
test_labels = app_test['TARGET']

In [4]:
from sklearn.preprocessing import MinMaxScaler, Imputer

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(['TARGET'], axis=1)
else:
    train = app_train.copy()

    
if 'TARGET' in app_test:
    test = app_test.drop(['TARGET'], axis=1)
else:
    test = app_test.copy()

    
# Feature names
features = list(train.columns)

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(test)

# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (215257, 449)
Testing data shape:  (46127, 449)


In [85]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn.utils.multiclass import type_of_target

import gc

def model(model_func, features, test_features, params, validation_features=None, n_folds = 5): 
    """Train, test and validation a light gradient boosting model using cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        
    Return
    --------
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    if validation_features is not None:
        validation_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = np.array(features['TARGET'].astype(int))
    test_labels = np.array(test_features['TARGET'].astype(int))
    if validation_features is not None:
        validation_labels = np.array(validation_features['TARGET'].astype(int))
    
    # Remove the ids and target
    features = features.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    test_features = test_features.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    if validation_features is not None:
        validation_features = validation_features.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    if validation_features is not None:
        print('Validation Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    if validation_features is not None:
        validation_features = np.array(validation_features)
        
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    if validation_features is not None:
        validation_predictions = np.zeros(validation_features.shape[0])
        
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores_auc = []
    train_scores_auc = []
    valid_scores_mae = []
    train_scores_mae = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        valid_score_auc, train_score_auc, valid_score_mae, train_score_mae, feature_importance_values_l, test_predictions_l, validation_predictions_l = model_func(train_features, train_labels, valid_features, valid_labels, test_features, validation_features, params)

        # Record the feature importances
        feature_importance_values += feature_importance_values_l / k_fold.n_splits

        # Make predictions
        test_predictions += test_predictions_l / k_fold.n_splits
        if validation_features is not None:
            validation_predictions += validation_predictions_l / k_fold.n_splits
        
        valid_scores_auc.append(valid_score_auc)
        train_scores_auc.append(train_score_auc)
        valid_scores_mae.append(valid_score_mae)
        train_scores_mae.append(train_score_mae)
        
        # Clean up memory
        gc.enable()
        del train_features, valid_features
        gc.collect()
        
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(test_labels, test_predictions)
    valid_mae = mean_absolute_error(test_labels, test_predictions)
    if validation_features is not None:
        validation_auc = roc_auc_score(validation_labels, validation_predictions)
        validation_mae = mean_absolute_error(validation_labels, validation_predictions)
    
    # Add the overall scores to the metrics
    valid_scores_auc.append(valid_auc)
    train_scores_auc.append(np.mean(train_scores_auc))
    valid_scores_mae.append(valid_mae)
    train_scores_mae.append(np.mean(train_scores_mae))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train auc': train_scores_auc,
                            'valid auc': valid_scores_auc,
                            'train mae': train_scores_mae,
                            'valid mae': valid_scores_mae}) 

    if validation_features is not None:
        validation_metrics = pd.DataFrame({'auc': [validation_auc],
                                           'mae': [validation_mae]}) 
    else:
        validation_metrics = None
        

    return feature_importances, metrics, validation_metrics

In [101]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from sklearn import metrics
from keras import backend as K
import tensorflow as tf

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

def train_NNClassifier(train_features, train_labels, valid_features, valid_labels, test_features, validation_features, params):
    # Create the model
    print( 'Setting up neural network...' )
    nn = Sequential()
    nn.add(Dense(units = params['layer1'] , input_dim = 448))
    nn.add(PReLU())
    nn.add(Dropout(params['dropout']))
    nn.add(Dense(units = params['layer2'] ))
    nn.add(PReLU())
    nn.add(BatchNormalization())
    nn.add(Dropout(params['dropout']))
    nn.add(Dense(units = params['layer3']))
    nn.add(PReLU())
    nn.add(BatchNormalization())
    nn.add(Dropout(params['dropout']))
    nn.add(Dense(units = params['layer4']))
    nn.add(PReLU())
    nn.add(BatchNormalization())
    nn.add(Dropout(params['dropout']))
    nn.add(Dense(units = params['layer5']))
    nn.add(PReLU())
    nn.add(BatchNormalization())
    nn.add(Dropout(params['dropout']))
    nn.add(Dense(1, activation='sigmoid'))
    nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mae', auc])
    
    # Callbacks
    callbacks = [EarlyStopping(monitor='val_loss', patience=100)]

    # Train the model
    nn.fit(train_features, train_labels, 
           validation_data=(valid_features, valid_labels), 
           epochs=params['epochs'], 
           callbacks=callbacks,
           verbose=2)

    # Record the feature importances
    feature_importance_values = np.zeros(train_features.shape[1])
    
    # Make predictions
    test_predictions = nn.predict(test_features).flatten().clip(0,1)
    if validation_features is not None:
        validation_predictions = nn.predict(validation_features).flatten().clip(0,1) 
    else:
        validation_predictions=None
        
    # Record the best score
    t_p = nn.predict(train_features).flatten().clip(0,1)
    v_p = nn.predict(valid_features).flatten().clip(0,1)
    
    # Record the best score
    valid_score_auc = roc_auc_score(valid_labels, v_p)
    train_score_auc = roc_auc_score(train_labels, t_p)
    valid_score_mae = mean_absolute_error(valid_labels, v_p)
    train_score_mae = mean_absolute_error(train_labels, t_p)

    return valid_score_auc, train_score_auc, valid_score_mae, train_score_mae, feature_importance_values, test_predictions, validation_predictions


In [52]:
def plot_feature_importances(df):
    """
    Plot importances returned by a model. This can work with any measure of
    feature importance provided that higher importance is better. 
    
    Args:
        df (dataframe): feature importances. Must have the features in a column
        called `features` and the importances in a column called `importance
        
    Returns:
        shows a plot of the 15 most importance features
        
        df (dataframe): feature importances sorted by importance (highest to lowest) 
        with a column for normalized importance
        """
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    return df

In [ ]:
params = {
    "layer1": 400,
    "layer2": 160,
    "layer3": 64,
    "layer4": 26,
    "layer5": 12,
    "epochs": 1000,
    "dropout": 0.3
}

fi, metrics, validation_metrics = model(train_NNClassifier, app_train, app_test, params=params)
print('Baseline metrics')
print(metrics)


Training Data Shape:  (215257, 448)
Testing Data Shape:  (46127, 448)
Setting up neural network...
Train on 172205 samples, validate on 43052 samples
Epoch 1/1000
 - 81s - loss: 0.3153 - mean_absolute_error: 0.1751 - auc: 0.5043 - val_loss: 0.2795 - val_mean_absolute_error: 0.1553 - val_auc: 0.5010
Epoch 2/1000
 - 75s - loss: 0.2841 - mean_absolute_error: 0.1522 - auc: 0.5006 - val_loss: 0.2783 - val_mean_absolute_error: 0.1500 - val_auc: 0.5024
Epoch 3/1000
 - 75s - loss: 0.2820 - mean_absolute_error: 0.1506 - auc: 0.5047 - val_loss: 0.2782 - val_mean_absolute_error: 0.1522 - val_auc: 0.5075
Epoch 4/1000
 - 75s - loss: 0.2813 - mean_absolute_error: 0.1502 - auc: 0.5100 - val_loss: 0.2779 - val_mean_absolute_error: 0.1492 - val_auc: 0.5117
Epoch 5/1000
 - 75s - loss: 0.2806 - mean_absolute_error: 0.1495 - auc: 0.5140 - val_loss: 0.2776 - val_mean_absolute_error: 0.1443 - val_auc: 0.5160
Epoch 6/1000
 - 76s - loss: 0.2805 - mean_absolute_error: 0.1494 - auc: 0.5176 - val_loss: 0.2775 - 

Epoch 55/1000
 - 76s - loss: 0.2771 - mean_absolute_error: 0.1476 - auc: 0.5749 - val_loss: 0.2749 - val_mean_absolute_error: 0.1460 - val_auc: 0.5751
Epoch 56/1000
 - 75s - loss: 0.2771 - mean_absolute_error: 0.1476 - auc: 0.5753 - val_loss: 0.2750 - val_mean_absolute_error: 0.1442 - val_auc: 0.5754
Epoch 57/1000
 - 76s - loss: 0.2770 - mean_absolute_error: 0.1474 - auc: 0.5756 - val_loss: 0.2751 - val_mean_absolute_error: 0.1444 - val_auc: 0.5758
Epoch 58/1000
 - 75s - loss: 0.2771 - mean_absolute_error: 0.1477 - auc: 0.5760 - val_loss: 0.2751 - val_mean_absolute_error: 0.1497 - val_auc: 0.5761
Epoch 59/1000
 - 75s - loss: 0.2769 - mean_absolute_error: 0.1474 - auc: 0.5763 - val_loss: 0.2754 - val_mean_absolute_error: 0.1494 - val_auc: 0.5765
Epoch 60/1000
 - 75s - loss: 0.2768 - mean_absolute_error: 0.1474 - auc: 0.5767 - val_loss: 0.2755 - val_mean_absolute_error: 0.1522 - val_auc: 0.5768
Epoch 61/1000
 - 75s - loss: 0.2771 - mean_absolute_error: 0.1476 - auc: 0.5770 - val_loss: 0.

Epoch 110/1000
 - 76s - loss: 0.2750 - mean_absolute_error: 0.1469 - auc: 0.5905 - val_loss: 0.2743 - val_mean_absolute_error: 0.1456 - val_auc: 0.5907
Epoch 111/1000
 - 75s - loss: 0.2751 - mean_absolute_error: 0.1471 - auc: 0.5908 - val_loss: 0.2741 - val_mean_absolute_error: 0.1445 - val_auc: 0.5909
Epoch 112/1000
 - 75s - loss: 0.2751 - mean_absolute_error: 0.1470 - auc: 0.5910 - val_loss: 0.2749 - val_mean_absolute_error: 0.1489 - val_auc: 0.5911
Epoch 113/1000
 - 75s - loss: 0.2751 - mean_absolute_error: 0.1470 - auc: 0.5912 - val_loss: 0.2743 - val_mean_absolute_error: 0.1449 - val_auc: 0.5913
Epoch 114/1000
 - 76s - loss: 0.2749 - mean_absolute_error: 0.1468 - auc: 0.5914 - val_loss: 0.2743 - val_mean_absolute_error: 0.1453 - val_auc: 0.5915
Epoch 115/1000
 - 75s - loss: 0.2751 - mean_absolute_error: 0.1470 - auc: 0.5916 - val_loss: 0.2742 - val_mean_absolute_error: 0.1465 - val_auc: 0.5917
Epoch 116/1000
 - 75s - loss: 0.2749 - mean_absolute_error: 0.1471 - auc: 0.5918 - val_l

Epoch 164/1000
 - 74s - loss: 0.2735 - mean_absolute_error: 0.1464 - auc: 0.6001 - val_loss: 0.2747 - val_mean_absolute_error: 0.1453 - val_auc: 0.6002
Epoch 165/1000
 - 74s - loss: 0.2735 - mean_absolute_error: 0.1464 - auc: 0.6002 - val_loss: 0.2749 - val_mean_absolute_error: 0.1422 - val_auc: 0.6003
Epoch 166/1000
 - 74s - loss: 0.2738 - mean_absolute_error: 0.1466 - auc: 0.6004 - val_loss: 0.2748 - val_mean_absolute_error: 0.1420 - val_auc: 0.6005
Epoch 167/1000
 - 74s - loss: 0.2736 - mean_absolute_error: 0.1464 - auc: 0.6005 - val_loss: 0.2748 - val_mean_absolute_error: 0.1407 - val_auc: 0.6006
Epoch 168/1000
 - 74s - loss: 0.2733 - mean_absolute_error: 0.1465 - auc: 0.6007 - val_loss: 0.2748 - val_mean_absolute_error: 0.1413 - val_auc: 0.6007
Epoch 169/1000
 - 74s - loss: 0.2734 - mean_absolute_error: 0.1463 - auc: 0.6008 - val_loss: 0.2745 - val_mean_absolute_error: 0.1437 - val_auc: 0.6009
Epoch 170/1000
 - 74s - loss: 0.2734 - mean_absolute_error: 0.1465 - auc: 0.6010 - val_l

Epoch 7/1000
 - 74s - loss: 0.2795 - mean_absolute_error: 0.1484 - auc: 0.5184 - val_loss: 0.2808 - val_mean_absolute_error: 0.1501 - val_auc: 0.5202
Epoch 8/1000
 - 74s - loss: 0.2793 - mean_absolute_error: 0.1484 - auc: 0.5217 - val_loss: 0.2809 - val_mean_absolute_error: 0.1462 - val_auc: 0.5232
Epoch 9/1000
 - 74s - loss: 0.2791 - mean_absolute_error: 0.1481 - auc: 0.5246 - val_loss: 0.2803 - val_mean_absolute_error: 0.1485 - val_auc: 0.5263
Epoch 10/1000
 - 74s - loss: 0.2789 - mean_absolute_error: 0.1482 - auc: 0.5277 - val_loss: 0.2803 - val_mean_absolute_error: 0.1476 - val_auc: 0.5292
Epoch 11/1000
 - 74s - loss: 0.2789 - mean_absolute_error: 0.1479 - auc: 0.5304 - val_loss: 0.2802 - val_mean_absolute_error: 0.1481 - val_auc: 0.5317
Epoch 12/1000
 - 74s - loss: 0.2786 - mean_absolute_error: 0.1478 - auc: 0.5329 - val_loss: 0.2803 - val_mean_absolute_error: 0.1499 - val_auc: 0.5341
Epoch 13/1000
 - 74s - loss: 0.2784 - mean_absolute_error: 0.1479 - auc: 0.5354 - val_loss: 0.280

...

Epoch 163/1000
 - 74s - loss: 0.2747 - mean_absolute_error: 0.1472 - auc: 0.5983 - val_loss: 0.2725 - val_mean_absolute_error: 0.1429 - val_auc: 0.5984
Epoch 164/1000
 - 74s - loss: 0.2744 - mean_absolute_error: 0.1469 - auc: 0.5984 - val_loss: 0.2725 - val_mean_absolute_error: 0.1398 - val_auc: 0.5985
Epoch 165/1000
 - 74s - loss: 0.2747 - mean_absolute_error: 0.1470 - auc: 0.5986 - val_loss: 0.2726 - val_mean_absolute_error: 0.1414 - val_auc: 0.5986
Epoch 166/1000
 - 74s - loss: 0.2747 - mean_absolute_error: 0.1470 - auc: 0.5987 - val_loss: 0.2726 - val_mean_absolute_error: 0.1437 - val_auc: 0.5988
Epoch 167/1000
 - 74s - loss: 0.2746 - mean_absolute_error: 0.1469 - auc: 0.5988 - val_loss: 0.2727 - val_mean_absolute_error: 0.1481 - val_auc: 0.5989
Epoch 168/1000
 - 74s - loss: 0.2745 - mean_absolute_error: 0.1471 - auc: 0.5990 - val_loss: 0.2727 - val_mean_absolute_error: 0.1449 - val_auc: 0.5990
Epoch 169/1000
 - 74s - loss: 0.2746 - mean_absolute_error: 0.1471 - auc: 0.5991 - val_l

Epoch 217/1000
 - 74s - loss: 0.2732 - mean_absolute_error: 0.1467 - auc: 0.6049 - val_loss: 0.2727 - val_mean_absolute_error: 0.1438 - val_auc: 0.6050
Epoch 218/1000
 - 74s - loss: 0.2734 - mean_absolute_error: 0.1466 - auc: 0.6050 - val_loss: 0.2725 - val_mean_absolute_error: 0.1407 - val_auc: 0.6051
Epoch 219/1000
 - 74s - loss: 0.2733 - mean_absolute_error: 0.1466 - auc: 0.6051 - val_loss: 0.2734 - val_mean_absolute_error: 0.1471 - val_auc: 0.6052
Epoch 220/1000
 - 74s - loss: 0.2731 - mean_absolute_error: 0.1467 - auc: 0.6052 - val_loss: 0.2732 - val_mean_absolute_error: 0.1400 - val_auc: 0.6053
Epoch 221/1000
 - 74s - loss: 0.2734 - mean_absolute_error: 0.1466 - auc: 0.6053 - val_loss: 0.2732 - val_mean_absolute_error: 0.1411 - val_auc: 0.6054
Epoch 222/1000
 - 74s - loss: 0.2735 - mean_absolute_error: 0.1466 - auc: 0.6054 - val_loss: 0.2726 - val_mean_absolute_error: 0.1453 - val_auc: 0.6055
Epoch 223/1000
 - 74s - loss: 0.2734 - mean_absolute_error: 0.1468 - auc: 0.6055 - val_l

Epoch 41/1000
 - 74s - loss: 0.2770 - mean_absolute_error: 0.1472 - auc: 0.5693 - val_loss: 0.2786 - val_mean_absolute_error: 0.1484 - val_auc: 0.5696
Epoch 42/1000
 - 74s - loss: 0.2769 - mean_absolute_error: 0.1470 - auc: 0.5698 - val_loss: 0.2781 - val_mean_absolute_error: 0.1501 - val_auc: 0.5700
Epoch 43/1000
 - 74s - loss: 0.2770 - mean_absolute_error: 0.1476 - auc: 0.5703 - val_loss: 0.2783 - val_mean_absolute_error: 0.1482 - val_auc: 0.5705
Epoch 44/1000
 - 74s - loss: 0.2771 - mean_absolute_error: 0.1474 - auc: 0.5707 - val_loss: 0.2779 - val_mean_absolute_error: 0.1459 - val_auc: 0.5709
Epoch 45/1000
 - 74s - loss: 0.2768 - mean_absolute_error: 0.1471 - auc: 0.5711 - val_loss: 0.2787 - val_mean_absolute_error: 0.1492 - val_auc: 0.5714
Epoch 46/1000
 - 74s - loss: 0.2769 - mean_absolute_error: 0.1474 - auc: 0.5716 - val_loss: 0.2778 - val_mean_absolute_error: 0.1478 - val_auc: 0.5718
Epoch 47/1000
 - 74s - loss: 0.2769 - mean_absolute_error: 0.1473 - auc: 0.5720 - val_loss: 0.

Epoch 96/1000
 - 74s - loss: 0.2752 - mean_absolute_error: 0.1467 - auc: 0.5857 - val_loss: 0.2774 - val_mean_absolute_error: 0.1437 - val_auc: 0.5859
Epoch 97/1000
 - 74s - loss: 0.2752 - mean_absolute_error: 0.1467 - auc: 0.5860 - val_loss: 0.2775 - val_mean_absolute_error: 0.1467 - val_auc: 0.5861
Epoch 98/1000
 - 74s - loss: 0.2753 - mean_absolute_error: 0.1470 - auc: 0.5862 - val_loss: 0.2777 - val_mean_absolute_error: 0.1418 - val_auc: 0.5863
Epoch 99/1000
 - 74s - loss: 0.2754 - mean_absolute_error: 0.1468 - auc: 0.5864 - val_loss: 0.2778 - val_mean_absolute_error: 0.1431 - val_auc: 0.5865
Epoch 100/1000
 - 74s - loss: 0.2752 - mean_absolute_error: 0.1466 - auc: 0.5866 - val_loss: 0.2788 - val_mean_absolute_error: 0.1431 - val_auc: 0.5867
Epoch 101/1000
 - 74s - loss: 0.2750 - mean_absolute_error: 0.1467 - auc: 0.5868 - val_loss: 0.2776 - val_mean_absolute_error: 0.1433 - val_auc: 0.5869
Epoch 102/1000
 - 74s - loss: 0.2750 - mean_absolute_error: 0.1465 - auc: 0.5870 - val_loss:

Epoch 150/1000
 - 74s - loss: 0.2736 - mean_absolute_error: 0.1462 - auc: 0.5956 - val_loss: 0.2773 - val_mean_absolute_error: 0.1447 - val_auc: 0.5957
Epoch 151/1000
 - 74s - loss: 0.2735 - mean_absolute_error: 0.1461 - auc: 0.5958 - val_loss: 0.2769 - val_mean_absolute_error: 0.1457 - val_auc: 0.5958
Epoch 152/1000


In [93]:
params = {
    "layer1": 800,
    "layer2": 400,
    "layer3": 200,
    "layer4": 80,
    "layer5": 20,
    "epochs": 1,
    "dropout": 0.3
}

fi, metrics, validation_metrics = model(train_NNClassifier, app_train, app_test, params=params)
print('Baseline metrics')
print(metrics)

Training Data Shape:  (215257, 448)
Testing Data Shape:  (46127, 448)
Setting up neural network...
448
Train on 172205 samples, validate on 43052 samples
Epoch 1/1
 - 75s - loss: 0.3142 - mean_absolute_error: 0.1745 - val_loss: 0.2788 - val_mean_absolute_error: 0.1466
Setting up neural network...
448
Train on 172205 samples, validate on 43052 samples
Epoch 1/1
 - 73s - loss: 0.3155 - mean_absolute_error: 0.1729 - val_loss: 0.2824 - val_mean_absolute_error: 0.1549
Setting up neural network...
448
Train on 172206 samples, validate on 43051 samples
Epoch 1/1
 - 76s - loss: 0.3132 - mean_absolute_error: 0.1754 - val_loss: 0.2769 - val_mean_absolute_error: 0.1527
Setting up neural network...
448
Train on 172206 samples, validate on 43051 samples
Epoch 1/1
 - 76s - loss: 0.3158 - mean_absolute_error: 0.1746 - val_loss: 0.2815 - val_mean_absolute_error: 0.1471
Setting up neural network...
448
Train on 172206 samples, validate on 43051 samples
Epoch 1/1
 - 77s - loss: 0.3157 - mean_absolute_er